In [191]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
from scipy.spatial.distance import pdist, cdist

In [206]:
def geo_distance(c_i, c_j):
    print(c_i, c_j)
    lng1, lat1 = c_i[0], c_i[1]
    lng2, lat2 = c_j[0], c_j[1]
    #lng1,lat1,lng2,lat2 = (120.12802999999997,30.28708,115.86572000000001,28.7427)
    lng1, lat1, lng2, lat2 = map(radians, [float(lng1), float(lat1), float(lng2), float(lat2)]) # 经纬度转换成弧度
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    distance=2*asin(sqrt(a))*6371*1000 # 地球平均半径，6371km
    distance=round(distance,3)
    return distance

In [168]:
def count_points(t):
    traj = t[0].split(' ')
    pad_cnt = traj.count('[PAD]')
    return len(traj) - pad_cnt

In [169]:
def rangeQuery(t, c):
    # 补一个token->(lon,lat)的逻辑
    cnt = 0
    coo_list = []
    traj = t[0].split(' ')

    for i in range(len(traj)):
        if traj[i] == '[PAD]':
            continue

        token = int(traj[i])
        print(token)
        coo_list.append((loc_index.iloc[token][1], loc_index.iloc[token][0]))

    for coo in coo_list:
        distance = geo_distance(coo, (c[0], c[1]))
        if distance < c[2]:
            cnt += 1
    return cnt

In [170]:
loc_index = pd.read_csv('./loc_index.csv',index_col=0)
all_traj = pd.read_hdf('./all_traj.h5', key='data')
cluster_center = pd.read_csv('../data/cluster_center.csv',header=None)
cluster_center.columns = ['lon','lat']

In [171]:
sgm = 0.6
lbd = 0.7
radius = pdist(cluster_center.values.tolist(), lambda c_i, c_j: geo_distance(c_i, c_j)).min()
cluster_center['radius'] = radius * sgm

In [184]:
cluster_center

,lon,lat,radius
0,116.319638,40.008637,688.9356
1,116.419351,39.969406,688.9356
2,116.351977,39.985927,688.9356
3,116.327723,39.979732,688.9356
4,116.416656,39.996251,688.9356
5,116.327723,39.996251,688.9356
6,116.448995,39.987992,688.9356
7,116.400486,40.051971,688.9356
8,116.303468,39.987992,688.9356
9,116.300773,39.977667,688.9356


In [188]:
loc_index = loc_index[['lon','lat']]


,lon,lat
0,116.830053,39.442088
1,116.830063,39.442098
2,116.643245,39.442125
3,116.829913,39.442126
4,116.323205,39.442175
...,...,...
69418,121.276390,41.058813
69419,117.987130,41.058855
69420,117.976205,41.058857
69421,117.803822,41.058894


In [204]:
cluster_center

,lon,lat,radius
0,116.319638,40.008637,688.9356
1,116.419351,39.969406,688.9356
2,116.351977,39.985927,688.9356
3,116.327723,39.979732,688.9356
4,116.416656,39.996251,688.9356
5,116.327723,39.996251,688.9356
6,116.448995,39.987992,688.9356
7,116.400486,40.051971,688.9356
8,116.303468,39.987992,688.9356
9,116.300773,39.977667,688.9356


In [203]:
loc_index

,lon,lat
0,116.830053,39.442088
1,116.830063,39.442098
2,116.643245,39.442125
3,116.829913,39.442126
4,116.323205,39.442175
...,...,...
69418,121.276390,41.058813
69419,117.987130,41.058855
69420,117.976205,41.058857
69421,117.803822,41.058894


In [207]:
geo_distance(loc_index.values[0],cluster_center.values[0])

[116.830053  39.442088] [116.31963809  40.00863696 688.9356    ]


76642.558

In [195]:
locs = loc_index.values.tolist()
centers = cluster_center[['lon','lat']].values.tolist()
dm = cdist(locs, centers, lambda loc, center: geo_distance(loc, center))

In [201]:
dm = pd.DataFrame(dm, columns=[('C_'+str(i))for i in range(len(cluster_center))])

In [214]:
loc_index['lon'].values.max()

133.598212

In [174]:
label = len(all_traj)*[-1]
for j, t in enumerate(all_traj.values):
    FR = []
    for i, c in enumerate(cluster_center.values):
        fallenPoints = rangeQuery(t, c)
        fallenRate = fallenPoints/count_points(t)
        FR.append((fallenRate,i))
        break
    FR.sort(key=lambda x: x[0], reverse=True)
    label[j] = FR[0][1]
all_traj['label'] = label

54062
55018
50219
49467
51143
56915
51959
50629
54874
56352
45764
45789
45814
52989
55120
54815
54645
56491
51123
51236
39161
28144
65754
26415
45790
56677
56145
56489
55460
54833
56502
56623
56386
56599
56507
56288
56423
56409
56278
56398
56243
56209
56436
56260
56383
56361
56195
56389
56181
56394
56551
56467
56474
56411
56492
56529
56359
56506
56553
56574
55726
50006
49524
44970
44987
49762
49716
52678
27124
8140
8414
8437
9186
5840
14758
55607
56329
55708
54793
52008
46885
17801
14766
14304
13684
13700
13317
14342
14737
13977
12651
12243
30940
55344
56311
55712
39319
60710
65875
65896
24173
39801
55352
54675
54673
53687
45411
46883
46954
51756
57262
54402
50150
52239
48970
47626
1648
835
819
2273
46187
56921
47898
10599
781
813
13491
47825
50602
55504
53618
50236
30936
7602
7579
8197
16031
51905
54420
56560
50801
4142
2817
4513
8170
14882
14870
16518
12993
7796
30672
46893
50598
56672
54078
53800
53925
54554
57361
53601
46155
46370
50916
53658
56177
51789
55489
53996
7536
7223
7029


In [176]:
all_traj['label'].max()

0

In [104]:
FR = [(0.9, 1),(0.8, 2),(0.7,3)]
FR.sort(key=lambda x: x[0], reverse=True)

In [111]:
t = all_traj.values[0]
print(t[0].split(' ').count('[PAD]'))

35


In [115]:
label = len(all_traj)*[-1]

In [117]:
all_traj['label'] = label
all_traj

,trajectory,user_index,year,month,day,label
0,930 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] ...,0,2009,3,31,-1
1,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,3,-1
2,237 236 234 233 210 209 186 183 179 178 172 17...,0,2009,4,4,-1
3,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,5,-1
4,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,6,-1
...,...,...,...,...,...,...
1890,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,13,-1
1891,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,15,-1
1892,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,2,-1
1893,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,12,-1
